In [77]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

from functools import reduce


from tqdm import tqdm, tqdm_notebook

from wavefunction1d import solve_eigenproblem

pi = np.pi

In [8]:
# QuTip
from qutip import Options, Qobj, basis, destroy, create, qeye, tensor, Cubic_Spline
from scipy.sparse import diags

In [11]:
Sx       = 101 
mformat  = 'csr'
dtype    = 'complex128'
x_pts = np.linspace(-100, 100, Sx, endpoint=True, dtype=dtype)


def build_operator(operator_name='idx', Dx=None):
    
    """
    Returns sparse operators in phase space. 
    """

    x_pts = np.linspace(-Dx, Dx, Sx, endpoint=True, dtype=dtype)
    dx    = x_pts[-1] - x_pts[-2] 
    
    if operator_name == 'idx':
        op = Qobj(diags(np.ones(x_pts.size), 0, shape=(Sx,Sx), format=mformat, dtype=dtype))
    
    elif operator_name == 'd1x':
        d1_coeff = (1.0 / (2.0 * dx))
        op       = Qobj(diags([-d1_coeff, d1_coeff], [-1,1], shape=(Sx,Sx), format=mformat, dtype=dtype))
    
    elif operator_name == 'd2x':
        d2_coeff = (1.0 / (dx**2))
        op       = Qobj(diags([d2_coeff, -2.0 * d2_coeff, d2_coeff], [-1,0,1], shape=(Sx,Sx), format=mformat, dtype=dtype))
    
    elif operator_name == 'x':
        op = Qobj(diags(x_pts, 0, shape=(Sx,Sx), format=mformat, dtype=dtype))
    
    elif operator_name == 'x2':
        op = Qobj(diags(x_pts**2, 0, shape=(Sx,Sx), format=mformat, dtype=dtype))
    
    elif operator_name == 'cos(x)':
        op = Qobj(diags(np.cos(x_pts), 0, shape=(Sx,Sx), format=mformat, dtype=dtype))
    
    elif operator_name == 'sin(x)':
        op = Qobj(diags(np.sin(x_pts), 0, shape=(Sx,Sx), format=mformat, dtype=dtype))

    elif operator_name == 'cos(x/2)':
        op = Qobj(diags(np.cos(x_pts/2.), 0, shape=(Sx,Sx), format=mformat, dtype=dtype))
    
    elif operator_name == 'sin(x/2)':
        op = Qobj(diags(np.sin(x_pts/2.), 0, shape=(Sx,Sx), format=mformat, dtype=dtype))

    return op

In [64]:
Ec = 30
EL = 0.5
Ej = 10

Nφ = 6

phi_max, Nphi = 10*np.pi, 501
phi_arr = np.linspace(-phi_max, phi_max, Nphi)

dims = [2*Nφ + 1, Nphi]

dphi = 2*phi_max/Nphi 

idx = Qobj(diags(np.ones(Nphi), 0, shape=(Nphi,Nphi), format=mformat, dtype=dtype))

phi = Qobj(diags(phi_arr, 0, shape=(Nphi,Nphi), format=mformat, dtype=dtype))

cos_phi = Qobj(diags(np.cos(phi_arr), 0, shape=(Nphi,Nphi), format=mformat, dtype=dtype))

def phi2 (phi_ext = 0):
    return Qobj(diags((phi_arr - phi_ext)**2, 0, shape=(Nphi,Nphi), format=mformat, dtype=dtype))

d2_coeff = (1.0 / (dphi**2))
d2phi = Qobj(diags([d2_coeff, -2.0 * d2_coeff, d2_coeff], [-1,0,1], 
                   shape=(Nphi,Nphi), format=mformat, dtype=dtype))

nφ   = Qobj(diags(np.arange(-Nφ,Nφ+1,1), 0, shape=(2*Nφ + 1, 2*Nφ + 1), format='csr', dtype='complex128'))
cosφ = Qobj(diags([0.5,0.5], [-1,1], shape=(2*Nφ + 1, 2*Nφ + 1), format='csr', dtype='complex128'))
sinφ = Qobj(diags([-.5j,.5j], [-1,1], shape=(2*Nφ + 1, 2*Nφ + 1), format='csr', dtype='complex128'))

cosφ

Quantum object: dims = [[13], [13]], shape = (13, 13), type = oper, isherm = True
Qobj data =
[[0.  0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.5 0.  0.5 0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.5 0.  0.5 0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.5 0.  0.5 0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.5 0.  0.5 0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.5 0.  0.5 0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.  0.5 0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.  0.5 0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.  0.5]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0. ]]

In [68]:


def flux_ops (**params):
    op_dict = {
    'x'      : Qobj(diags(phi_arr, 0, shape=(Nphi,Nphi), format=mformat, dtype=dtype)),
    'x2'     : Qobj(diags((phi_arr - phi_ext)**2, 0, shape=(Nphi,Nphi), format=mformat, dtype=dtype)),
    'cos(x)' : Qobj(diags(np.cos(phi_arr), 0, shape=(Nphi,Nphi), format=mformat, dtype=dtype))
    }
    return op_dict


def charge_ops (**params):
    op_dict = {
    'nφ' : Qobj(diags(np.arange(-Nφ,Nφ+1,1), 0, shape=(2*Nφ + 1, 2*Nφ + 1), format='csr', dtype='complex128')),
    'cosφ' : Qobj(diags([0.5,0.5], [-1,1], shape=(2*Nφ + 1, 2*Nφ + 1), format='csr', dtype='complex128')),
    'sinφ' : Qobj(diags([-.5j,.5j], [-1,1], shape=(2*Nφ + 1, 2*Nφ + 1), format='csr', dtype='complex128'))
    }
    return op_dict


In [73]:
charge_ops()['nφ'].shape

(13, 13)

In [75]:
def compose(operator, index, dims):

    '''
    Returns the operator composed in a multiqubit Hilbert space.
    '''

    op_list        = [qeye(dim) for dim in dims]
    op_list[index] = operator

    return reduce(tensor, op_list)

In [76]:
def batch_compose(dicts, dims):
    
#     dims = []
    out = []
    for i, dct in enumerate(dicts):
        for key, val in dct.items():
            dct[key] =  compose(dct[key], i, dims)
        out.append(dct)
    return out

In [74]:
batch_compose(dicts, dims = )

NameError: name 'compose' is not defined

In [57]:
from  qutip.parallel import parallel_map
import time

In [61]:
def sweep(function, sweep_variable, sweep_vector, **params):

    '''
    Sweeps a function in a range of a scalar variable.
    '''

    if not sweep_variable in list(params.keys()): raise Exception('sweep_variable is not in params')

    sweep_results = []
    original_values = copy.deepcopy(params) # copy original params values
    for sweep_point in sweep_vector:
        params[sweep_variable] = sweep_point
        sweep_results.append(function(**params))
    params = copy.deepcopy(original_values) # restore params to the original values

    return {'result': sweep_results, 'variable': sweep_variable, 'vector': sweep_vector}

In [60]:
t = time.time()


def dev_diag(phi_ext = 0):
    H  = -Ec*d2phi
    H +=  Ej*cos_phi*
    H +=  EL*phi2(phi_ext)
    
    H +=  EL*phi2(phi_ext)

    evals, ekets = H.eigenstates()
    
    return evals, ekets

phi_exts = np.linspace(-3, 3, 51)

Es0 = []
Es1 = []

for phi_ext in phi_exts:
    evals, ekets = dev_diag(phi_ext)
    
    Es0.append(evals[0])
    Es1.append(evals[1])

fig, ax = plt.subplots()
ax.plot(phi_exts,Es0, 'o')

ax.plot(phi_exts,Es1, 'o')

print(time.time() - t)
# np.shape(psis)

FigureCanvasNbAgg()

17.859259843826294


In [62]:
sweep(dev_diag, phi_ext, np.linspace(-3, 3, 51))

Exception: sweep_variable is not in params

In [ ]:
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

In [ ]:
phi_max, Nphi = 10*np.pi, 501
phi = np.linspace(-phi_max, phi_max, Nphi)

dphi = 2*phi_max/Nphi 

K_mtx = np.diag( -2* np.ones(Nphi) ) + np.diag( np.ones(Nphi - 1 ) , 1) + np.diag( np.ones(Nphi - 1), -1)

Ec = 30
EL = 0.5
Ej = 10

phiext_list = np.linspace(0 , np.pi/2, 21)

psi0s = []
psi1s = []


V = Ej* np.cos(phi - 1*np.pi/4)  + EL*(phi)**2   

H = -Ec*K_mtx/dphi**2 + np.diag(V)

evals, evecs = solve_eigenproblem(H)

fig, ax = plt.subplots()


a = -50

psi0 = a*evecs[0] +  evals[0]
psi1 = a*evecs[1] +  evals[1]



ax.plot(phi, psi0)
ax.plot(phi, -psi1)

ax2 = ax.twinx()
# ax2.plot(phi[400:-400], V[400:-400], '--b', alpha = 0.2)
ax.plot(phi[abs(phi)<7], V[abs(phi)<7], '--b', alpha = 0.2)


## lambds scan

In [3]:
phi_max, Nphi = 10*np.pi, 501
phi = np.linspace(-phi_max, phi_max, Nphi)

dphi = 2*phi_max/Nphi 

K_mtx = np.diag( -2* np.ones(Nphi) ) + np.diag( np.ones(Nphi - 1 ) , 1) + np.diag( np.ones(Nphi - 1), -1)

Ec = 15
EL = 0.5
Ej = 15


Nframes = 201
phiext_list = np.linspace(-np.pi/2, np.pi/4, Nframes)

psi0s = []
psi1s = []

E0s = []
E1s = []

Vs = []

a = -50

tphiext_list = tqdm_notebook(phiext_list)

for phiext in tphiext_list:
    V = Ej* np.cos(phi - phiext)  + EL*(phi)**2   

    H = -Ec*K_mtx/dphi**2 + np.diag(V)

    evals, evecs = solve_eigenproblem(H)

    psi0 = -a*evecs[0] 
    psi1 = a*evecs[1] 

    E0 =  evals[0]
    E1 =  evals[1]

    
    psi0s.append(psi0)
    psi1s.append(psi1)

    E0s.append( E0 )
    E1s.append( E1 )
    
    Vs.append( V )
    

In [4]:
from matplotlib.animation import FuncAnimation, PillowWriter

fig, (ax, ax1) = plt.subplots(1,2, figsize=(7, 4), sharey = False)

fs = 20

ax.set_xticklabels([])
ax.set_yticklabels([])

ax.set_xlabel(r'$\phi$', fontsize = fs)
ax.set_ylabel(r'$\Psi(\phi)$', fontsize = fs)



ax1.set_yticks([])

ax2 = ax1.twinx()
ax1.set_xticklabels([])
ax2.set_yticklabels([])

ax1.set_xlabel(r'$\lambda$', fontsize = fs)
ax2.set_ylabel(r'$E_i$', fontsize = fs)


line1, = ax.plot(phi /2/np.pi,  psi0s[0] + E0s[0], c = 'C0', label = r'$|g\rangle$')
line2, = ax.plot(phi /2/np.pi,  psi1s[0] + E1s[0], c = 'C1', label = r'$|e\rangle$')


ax.legend(loc = 3, fontsize = 15)

ax2.plot(phiext_list, E0s , label = r'$E_g$' )
ax2.plot(phiext_list, E1s , label = r'$E_e$' )

ax2.legend(loc = 1, fontsize = 15)

m1, = ax2.plot([phiext_list[0]],  [E0s[0]], c = 'C0', marker = 'x')
m2, = ax2.plot([phiext_list[1]],  [E1s[1]], c = 'C1', marker = 'x')

line3, = ax.plot(phi /2/np.pi, Vs[0], '--k', alpha = 0.2)

signs = [np.sign(psi0s[0][Nphi//2+15]),
         np.sign(psi0s[1][Nphi//2+15])]

ax.set_xlim(-2.25,2.25)
ax.set_ylim(-20,25)

plt.tight_layout()

lines = [line1, line2, line3]

markers = [m1, m2]

psis = [psi0s, psi1s, Vs]

Es = [E0s, E1s]

def update(j):
    
    for l, ps,E, sign in zip(lines, psis,Es, signs):
        psi = np.array(ps[j])
        x = phi /2/pi

        y = 1*psi*sign*np.sign(psi[Nphi//2+15])
    
        l.set_data(x , y + E[j])
        
    lines[2].set_data(x , Vs[j])
    
    
    for m, E in zip(markers, Es):
         
        x = phiext_list[j]

        y = E[j]
    
        m.set_data(x , y)
     

    return lines, markers, ax, ax2


fig.savefig('wf_static.png')    
    
anim = FuncAnimation(fig, update, frames=np.arange(0, 2*Nframes//3), interval=20,repeat=False)

anim.save('wf.gif')

FigureCanvasNbAgg()

MovieWriter ffmpeg unavailable; trying to use <class 'matplotlib.animation.PillowWriter'> instead.


In [78]:
plt.close('all')

## EL scan

In [6]:
201//4

50

In [5]:
phi_max, Nphi = 10*np.pi, 201
phi = np.linspace(-phi_max, phi_max, Nphi)

dphi = 2*phi_max/Nphi 

K_mtx = np.diag( -2* np.ones(Nphi) ) + np.diag( np.ones(Nphi - 1 ) , 1) + np.diag( np.ones(Nphi - 1), -1)

Ec = 15
# EL = 0.5
phiext0 = -np.pi/4


Ej = 15


Nframes = 201


EL_list = np.logspace(-0.3, -3, Nframes)
phiext_list = np.linspace(-np.pi/2, np.pi/4, 101)

psi0s = []
psi1s = []

E0s = []
E1s = []



band0s = []
band1s = []

Vs = []

a = -50

tEL_list = tqdm_notebook(EL_list)

for EL in tEL_list:
    V = Ej* np.cos(phi - phiext0)  + EL*(phi)**2   

    H = -Ec*K_mtx/dphi**2 + np.diag(V)

    evals, evecs = solve_eigenproblem(H)

    psi0 = -a*evecs[0] 
    psi1 = a*evecs[1] 

    E0 =  evals[0]
    E1 =  evals[1]

    
    psi0s.append(psi0)
    psi1s.append(psi1)

    E0s.append( E0 )
    E1s.append( E1 )
    
    Vs.append( V )
    
    
    band0 = []
    band1 = []
    for phiext in phiext_list:
        V = Ej* np.cos(phi - phiext)  + EL*(phi)**2   

        H = -Ec*K_mtx/dphi**2 + np.diag(V)

        evals, evecs = solve_eigenproblem(H)
        
        band0.append(evals[0])
        band1.append(evals[1])
    
    band0s.append(band0)
    band1s.append(band1)


In [21]:
from matplotlib.animation import FuncAnimation, PillowWriter

fig, (ax, ax1) = plt.subplots(1,2, figsize=(7, 4), sharey = False)

fs = 20

ax.set_xticklabels([])
ax.set_yticklabels([])

ax.set_xlabel(r'$\phi$', fontsize = fs)
ax.set_ylabel(r'$\Psi(\phi)$', fontsize = fs)



ax1.set_yticks([])

ax2 = ax1.twinx()
ax1.set_xticklabels([])
ax2.set_yticklabels([])

ax1.set_xlabel(r'$\lambda$', fontsize = fs)
ax2.set_ylabel(r'$E_i$', fontsize = fs)

line1, = ax.plot(phi /2/np.pi,  psi0s[0] + 0*E0s[0], c = 'C0', label = r'$|g\rangle$')
line2, = ax.plot(phi /2/np.pi,  psi1s[0] + 0*E1s[0], c = 'C1', label = r'$|e\rangle$')

ax.legend(loc = 3, fontsize = 15)


line21, = ax2.plot(phiext_list, band0s[0] - band0s[0][0], label = r'$E_g$'  )
line22, = ax2.plot(phiext_list, band1s[1] - band0s[0][0], label = r'$E_e$' )

ax2.legend( loc = 1, fontsize = 15)

# ax2.set_title(r'$E_L = $' + '{:1.2g}'.format(EL_list[0]))

m1, = ax2.plot([phiext0],  [E0s[0] - band0s[0][0]], 'x', c = 'C0')
m2, = ax2.plot([phiext0],  [E1s[0] - band0s[0][0]], 'x', c = 'C1')

# m1, = ax2.plot([phiext_list[0]],  [E0s[0]], c = 'C0', marker = 'x')
# m2, = ax2.plot([phiext_list[1]],  [E1s[1]], c = 'C1', marker = 'x')

line3, = ax.plot(phi /2/np.pi, Vs[0] - E0s[0], '--k', alpha = 0.2)

signs = [np.sign(psi0s[0][Nphi//2+15]),
         -np.sign(psi0s[1][Nphi//2+15])]

ax.set_xlim(-2.25,2.25)
ax.set_ylim(-20,25)



plt.tight_layout()

lines = [line1, line2, line3]

line2s = [line21, line22]

markers = [m1, m2]

psis = [psi0s, psi1s, Vs]

Es = [E0s, E1s]

bands = [band0s, band1s]

def update(j):
    
    for l, ps,E, sign in zip(lines, psis,Es, signs):
        psi = np.array(ps[j])
        x = phi /2/pi

        y = 1*psi*sign*np.sign(psi[Nphi//2+15])
    
        l.set_data(x , y + 0*E[j])
        
    lines[2].set_data(x , Vs[j] - E[j])
    
    
    for l, band in zip(line2s, bands):
         
        x = phiext_list

        y = np.array(band[j]) - np.array(band0s)[j,0]
    
        l.set_data(x , y)

        
        
    for m, E in zip(markers, Es):
         
        x = [phiext0]

        y = [E[j] - band0s[j][0]]
    
        m.set_data(x , y)        
        
#     ax2.set_title(r'$E_L = $' + '{:1.2g}'.format(EL_list[j]))
    return lines, markers, ax, ax2


    
    
anim = FuncAnimation(fig, update, frames=np.arange(0, Nframes), interval=20, repeat=False)

anim.save('wf1.gif')

FigureCanvasNbAgg()

MovieWriter ffmpeg unavailable; trying to use <class 'matplotlib.animation.PillowWriter'> instead.
